In [2]:
import os
import copy
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import geojson
from geojson import LineString, FeatureCollection, Feature
from mapboxgl.utils import create_color_stops, create_numeric_stops
from shapely.geometry import shape, Polygon

from operator import itemgetter
from mapbox import DirectionsMatrix, Directions
from geojson import Point, Feature
from datetime import timedelta

num_holidays = 2
csv_path = './Florence.csv'
TRAVEL_NAME = os.path.split(csv_path)[-1].split(".")[0]
edge_path = "{}_edge_list.csv".format(TRAVEL_NAME)
start_point_info = {"name": "Airbnb: Via del Porcellana", "lat":43.772596, "long":11.247192}

mapbox_api_key = None

In [3]:
from duration_by_mapbox import MapboxDistance

md = MapboxDistance(mapbox_api_key)

In [4]:
from data_preparation import prepare_dataset, add_start_point, generate_edge_dataset

In [5]:
df = prepare_dataset(csv_path)

In [6]:
df.head()

,index,name,lat,long,hours,type
0,0,Cathedral of Santa Maria del Fiore,43.772858,11.255693,1.0,attraction
1,1,Chiaroscuro,43.771534,11.256419,0.5,cafee
2,2,Holy Trinity,43.770083,11.250781,0.5,attraction
3,3,Mercato di San Lorenzo,43.776919,11.253412,1.0,attraction
4,4,Villa Bardini,43.763469,11.255580,1.5,garden


In [8]:
from random import randint
from collections import Counter
from itertools import groupby

In [9]:
from sklearn.cluster import KMeans

latlon = np.array(df[['lat', 'long']].to_dict('split')['data'])
kmeans = KMeans(n_clusters=2).fit(latlon)

In [10]:
import collections

cluster_centers = kmeans.cluster_centers_

df['cluster_num'] = kmeans.labels_
count_per_cluster = dict(collections.Counter(kmeans.labels_))

clusterDF = pd.DataFrame.from_dict(data=cluster_centers)
clusterDF.columns=['latitude','longitude']  

In [11]:
df

,index,name,lat,long,hours,type,cluster_num
0,0,Cathedral of Santa Maria del Fiore,43.772858,11.255693,1.00,attraction,0
1,1,Chiaroscuro,43.771534,11.256419,0.50,cafee,0
2,2,Holy Trinity,43.770083,11.250781,0.50,attraction,0
3,3,Mercato di San Lorenzo,43.776919,11.253412,1.00,attraction,0
4,4,Villa Bardini,43.763469,11.255580,1.50,garden,0
5,5,Uffizi Gallery,43.767786,11.255311,0.50,attraction,0
6,6,Perch no!...,43.770828,11.255432,0.50,ice cream,0
7,7,Giotto's Bell Tower,43.772806,11.255705,0.75,attraction,0
8,8,Ponte Vecchio,43.767925,11.253144,0.20,attraction,0
9,9,Corridoio Vasariano,43.768191,11.254148,0.50,attraction,0


In [12]:
sum(df[df['cluster_num'] == 0]['hours'])

16.45

In [13]:
from mapboxgl.utils import create_color_stops, df_to_geojson
from mapboxgl.viz import CircleViz, LinestringViz

ACCESS_TOKEN = 'pk.eyJ1IjoiZmlvbmFjaG93IiwiYSI6ImNpdTg4NTNkODAwMGsydGxpdmZmYjk5YW0ifQ.e-fvB8_7eDtKx1iZCox7zg'

geojson_path = '{}.geojson'.format(TRAVEL_NAME)

# Create a geojson file export from a Pandas dataframe
df_to_geojson(df, filename=geojson_path,
              properties=["index", 'name', "cluster_num"],
              lat='lat', lon='long', precision=3)

from clustering import create_color_stops

color_stops = create_color_stops(df, 'cluster_num')
# Create the viz from the dataframe
viz = CircleViz(geojson_path,
                access_token=ACCESS_TOKEN,
                height='400px',
                center = (11.255705, 43.772806),
                zoom = 11,
                below_layer = 'waterway-label'
              )

In [14]:
#Marker color-related attributes
viz.color_property = 'cluster_num'
viz.color_function_type = 'match'
viz.color_stops = color_stops
viz.radius = 2

viz.show()

In [47]:
if os.path.exists(edge_path):
    edge_df = pd.read_csv(edge_path)
else:
    edge_df = generate_edge_dataset(df)

edge_df.tail()

,index,from,to,from_xy,to_xy,walking,driving,lat,long
177,177,-1,8,"11.251184, 43.770312","11.2531435, 43.767925",292.4,748.4,11.251184,43.770312
178,178,-1,9,"11.251184, 43.770312","11.2541477, 43.7681911",291.4,725.8,11.251184,43.770312
179,179,-1,10,"11.251184, 43.770312","11.250670199999998, 43.767161200000004",291.7,626.1,11.251184,43.770312
180,180,-1,11,"11.251184, 43.770312","11.2554649, 43.7730912",773.5,266.9,11.251184,43.770312
181,181,-1,12,"11.251184, 43.770312","11.24938, 43.77463",466.8,607.6,11.251184,43.770312


In [75]:
subdf

,index,name,lat,long,hours,cluster_num
0,0,Cathedral of Santa Maria del Fiore,43.772858,11.255693,1.00,0
1,1,Chiaroscuro,43.771534,11.256419,0.50,0
3,3,Mercato di San Lorenzo,43.776919,11.253412,1.00,0
6,6,Perch̩ no!...,43.770828,11.255432,0.50,0
7,7,Giotto's Bell Tower,43.772806,11.255705,0.75,0
11,11,Piazza del Duomo,43.773091,11.255465,1.50,0
12,12,Basilica of Santa Maria Novella,43.774630,11.249380,1.00,0


In [272]:
viz_by_grp = []
for grp, subdf in df.groupby(['cluster_num']):
    subdf.drop(['cluster_num'], axis=1, inplace=True)
    centerpoint = get_centerpoint(subdf)
    subdf = add_start_point(subdf, start_point_info=start_point_info)
    print("creating route for day {}".format(grp+1))
    attractions = list(subdf.index)
    print("Number of attractions: {}".format(len(attractions)-1))
    print(attractions)
    sub_edge_df = edge_df[edge_df['from'].isin(attractions) & edge_df['to'].isin(attractions)]
    g = draw_graph(sub_edge_df, subdf, directed=True)
    visit_path = brute_force_approach(g)
    viz = create_vis_object(g, centerpoint, visit_path, ACCESS_TOKEN)
    viz_by_grp.append([grp, viz])
    print("__________________________________________")
    print("__________________________________________")

creating route for day 1
Number of attractions: 7
[0, 1, 3, 6, 7, 11, 12, -1]
Total hours spent: 14.25
visiting: -1
visited:  [-1]
node 6 not visited before
visiting: 6
visited:  [-1, 6]
node 1 not visited before
visiting: 1
visited:  [-1, 6, 1]
node 0 not visited before
visiting: 0
visited:  [-1, 6, 1, 0]
node 7 not visited before
visiting: 7
visited:  [-1, 6, 1, 0, 7]
node 12 not visited before
visiting: 12
visited:  [-1, 6, 1, 0, 7, 12]
node 3 not visited before
visiting: 3
visited:  [-1, 6, 1, 0, 7, 12, 3]
node 11 not visited before
remaining unvisited: [-1]
Total travel time in hours: 0.7264722222222223
visit_path:  [[-1, 6], [6, 1], [1, 0], [0, 7], [7, 12], [12, 3], [3, 11], [11, -1]]
__________________________________________
__________________________________________
creating route for day 2
Number of attractions: 6
[2, 4, 5, 8, 9, 10, -1]
Total hours spent: 11.7
visiting: -1
visited:  [-1]
node 2 not visited before
visiting: 2
visited:  [-1, 2]
node 10 not visited before
visit

/Users/fionachow/Documents/Git/virtualenv_python3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/Users/fionachow/Documents/Python/airbnb/notebook/data_preparation.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [271]:
for vbg in viz_by_grp:
    vbg[1].show()

In [275]:
visits = [[[-1, 6, 1, 0, 7, 12, 3,11], [-1, 2, 10, 8, 9, 5, 4]]]

In [ ]:
for day in visits:
    

In [90]:
# from network_graph import draw_graph

In [268]:
def brute_force_approach(g):
    time_spent = sum([g.node[n]['hours'] for n in g.nodes])
    print("Total hours spent: {}".format(time_spent))

    length_dict=dict(nx.all_pairs_dijkstra_path_length(g))
    path_dict=dict(nx.all_pairs_dijkstra_path(g))

    start_point = -1
    current_point = start_point
    pending_visit = list(length_dict.keys())
    visited = [start_point]
    distance = []

    while len(pending_visit)>1:
        print("visiting: {}".format(current_point))
        print("visited: ", visited)
        current_point_lengths = length_dict[current_point]
        del current_point_lengths[current_point]
        for node, dist in current_point_lengths.items():
            if node not in visited:
                print("node {} not visited before".format(node))
                current_point = node
                visited.append(current_point)
                pending_visit.remove(current_point)
                distance.append(dist)
                break
    print("remaining unvisited: {}".format(pending_visit))
    visit_path = [path_dict[v][visited[e+1]] for e, v in enumerate(visited[:-1])]

    # last route back to start point
    visit_path.append(nx.dijkstra_path(g, visited[-1], start_point))
    distance.append(nx.dijkstra_path_length(g, visited[-1], start_point))

    print("Total travel time in hours: {}".format(sum(distance)/60/60))
    print("visit_path: ", visit_path)

    return visit_path

In [256]:
def get_centerpoint(df):
    points_lst = list(zip(df['long'], df['lat']))
    points_lst.append(points_lst[0])
    centerpoint = Polygon(points_lst).centroid
    return centerpoint

In [92]:
def draw_graph(edge_df, node_df, weight='walking', directed=False):
    # Create empty graph
    if directed:
        g = nx.DiGraph()
    else:
        g = nx.Graph()

    edge_col = edge_df.columns
    node_col = node_df.columns

    # Add edges and edge attributes
    for i, elrow in edge_df.iterrows():
        edge_dict = dict(zip(edge_col, elrow))
        g.add_edge(edge_dict['from'], edge_dict['to'], attr_dict=edge_dict, weight=edge_dict[weight])

    for i, nlrow in node_df.iterrows():
        node_dict = dict(zip(node_col, nlrow))
        g.node[node_dict['index']].update(node_dict)
    return g

In [258]:
def create_vis_object(g, centerpoint, visit_path, mapbox_token):
    feature_collection = []
    for e, vp in enumerate(visit_path):
        vp_linestring = LineString([(g.node[p]['long'], g.node[p]['lat']) for p in vp])
        from_to = " to ".join([g.node[p]['name'].replace("'", "") for p in vp])
        f = Feature(id=e+1, geometry=vp_linestring, properties={"walking_distance": distance[e], "from-to": from_to, "order": e+1})
        feature_collection.append(f)

    fc = FeatureCollection(feature_collection)
    viz = LinestringViz(fc, 
                    access_token=mapbox_token,
                    color_property='order',
                    color_stops= create_color_stops([i+1 for i in range(len(visit_path))], colors='YlOrRd'),
                    line_stroke='solid',
                    opacity=0.8,
                    line_width_default=2,
                    color_default="black",
                    line_width_property='walking_distance',
                    line_width_stops=create_numeric_stops([300, 600, 900], 2, 10),
                    height='400px',
                    center = (centerpoint.x, centerpoint.y),
                    zoom = 11,
                    below_layer='waterway-label')
    return viz